In [ ]:
%%time

## DATA IMPORT ## 
## For this project the data from https://www.kaggle.com/c/ieee-fraud-detection/data has been used. ##

data_path = "./data/"

train = pd.read_csv(data_path + "train_processed.csv")
test = pd.read_csv(data_path + "test_processed.csv")

print('train_set shape is: {}'.format(train.shape))
print('test_set shape is: {}'.format(test.shape))

In [ ]:
X = train.copy()
y = train['isFraud'].copy()
X = X.drop(['isFraud'], axis=1)

In [ ]:
import xgboost as xgb

data_dmatrix = xgb.DMatrix(data=X,label=y) # define data_dmatrix

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17, shuffle=False)

from sklearn.impute import SimpleImputer

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_train)

# Impute our data, then train
X_train_imp = imp.transform(X_train)

from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Undersample the train dataset
ros = RandomUnderSampler(random_state=17)

X_train, y_train = ros.fit_resample(X_train_imp, y_train)
print('Resampled dataset shape {}'.format(Counter(y_train)))



In [ ]:
# UNDERSAMPLING APPROACH - XGBOOST

from xgboost import XGBClassifier


# declare parameters
params = {
    'objective':'binary:logistic',
    'max_depth': 4,
    'alpha': 10,
    'learning_rate': 1.0,
    'n_estimators':100
}         
           
          
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)


# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

from sklearn.metrics import accuracy_score

print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
from xgboost import cv

params = {
    "objective":"binary:logistic",
    'colsample_bytree': 0.3,
    'learning_rate': 0.1,
    'max_depth': 5, 
    'alpha': 10}

xgb_cv = cv(dtrain=data_dmatrix,
            params=params, 
            nfold=3,
            num_boost_round=50, 
            early_stopping_rounds=10, 
            metrics="auc", 
            as_pandas=True,
            seed=37)

In [ ]:
xgb_cv.head()